# Pinhole Camera Model Practice Questions

## Question 1: Perspective Projection

A 3D point in the camera coordinate system is given by `p_c = [10, 5, 20]`. 
The camera has a focal length `f` of 50mm. 
Calculate the position of the projection of this point on the image plane in meters.

In [3]:
# Your code here
import numpy as np
def F15(pc,f):
    x, y, z = pc.flatten()
    u = (f * x) / z
    v = (f * y) / z
    return np.array([u, v])
F15X=print(F15(np.array([[10], [5], [20]]),0.05))

[0.025  0.0125]


---

## Question 2: Conversion to Pixels

Given the projected point from Question 1, convert its coordinates from meters to pixels. 
Assume the following intrinsic parameters: 
- `s_x` = 800 pixels/meter
- `s_y` = 800 pixels/meter
- `o_x` = 320 pixels
- `o_y` = 240 pixels
What are the pixel coordinates `(u^I, v^I)`?

In [5]:
# Your code here
def F47(u_m,v_m,s_x,s_y,o_x,o_y):
    u_I = s_x * u_m + o_x
    v_I = s_y * v_m + o_y
    return np.array([u_I, v_I])
print(F47(0.025,0.0125,800,800,320,240))

[340. 250.]


---

## Question 3: Intrinsic Matrix

Construct the intrinsic calibration matrix `K` using the parameters from Question 2, assuming no skew (`s_θ = 0`).

In [6]:
# Your code here
def F16(s_x,s_y,o_x,o_y,f):
    K = np.array([
        [s_x*f,   0,  o_x],
        [  0,  s_y*f, o_y],
        [  0,   0,   1 ]
    ])
    return K
print(F16(800,800,320,240,0.05))

[[ 40.   0. 320.]
 [  0.  40. 240.]
 [  0.   0.   1.]]


---

## Question 4: Point Projection with Known Pose

A point in the world frame is given by `p_w = [2, 3, 50]`. The camera's pose `T_w^c` is given by an identity rotation matrix and a translation vector `t_w^c = [0, 0, 10]`. [cite_start]Using the intrinsic matrix `K` from Question 3, calculate the pixel coordinates of the projected point. [cite: 14, 16, 17]

In [10]:
# Your code here
def F18F(p_w, t_wc, K):
    R_wc = np.eye(3)

    p_w_homog = np.append(p_w, 1)  # shape (4,)
    
    T_wc = np.hstack((R_wc, t_wc.reshape(3,1)))  # shape (3,4)

    p_c = np.matmul(T_wc,p_w_homog)  # shape (3,)

    p_img = np.matmul(K,p_c)  # shape (3,)
    
    u = p_img[0] / p_img[2]
    v = p_img[1] / p_img[2]
    k=p_img[2]/p_img[2]

    return np.array([u, v,k])
K=F16(800,800,320,240,0.05)
print(F18F(np.array([2, 3, 50]),np.array([0,0,10]),K))

[321.33333333 242.           1.        ]


---

## Question 5: Radial Distortion

A point is projected on the image plane at `(u_distort^I, v_distort^I) = (400, 350)` including radial distortion. Given distortion coefficients `a1 = 0.0001` and `a2 = 0.0000002` and the principal point `(o_x, o_y) = (320, 240)`, calculate the corrected pixel coordinates `(u^I, v^I)`.

In [9]:
# Your code here
def F22(u_distort,v_distort,o_x,o_y,a1,a2):
    dx = u_distort - o_x
    dy = v_distort - o_y

    # Compute r^2
    r2 = dx**2 + dy**2
    r4 = r2**2

    # Radial correction factor
    factor = 1 + a1 * r2 + a2 * r4

    # Apply distortion correction
    u_corrected = factor * dx + o_x
    v_corrected = factor * dy + o_y

    return np.array([u_corrected, v_corrected])
print(F22(400,350,320,240,0.0001,0.0000002))    

[6024. 8083.]
